# Clean extracted data
To do:
* Find Names
* Create function getAllCandidats

In [2]:
import json
import pandas as pd
from pathlib import Path
import re
import utils

In [3]:
# Get all files
df = pd.read_csv(Path('../export/dokumente.csv'))

# Only "Protokoll"
df = df[df.dokument_kategorie == 'Protokoll']

df_raete = pd.read_csv(Path('../export/mitglieder.csv'))
df_raete['_name'] = df_raete['vorname'] +' ' + df_raete['name']
df_raete['_name'] = df_raete['_name'].str.lower()


# Some names are written wrong. Some changed their name (marriage). Add
# Aliases to the dataframe

for a in utils.aliases:
    df_raete = df_raete.append({
        '_name': a[0].lower(),
        'name': a[2],
        'vorname': a[1]}, ignore_index=True)

In [4]:
# Remove this
remove = ['Ratsvizepräsidentin', 'Ratspräsidentin', 'Regierungsrätin',
     'Ratspräsident', 'Regierungsrat', 'Ratsvizepräsident', 
    ', Niederhasli.', ', SVP, Zürich.', ', SVP, Dinhard.',
    ', GLP, Dübendorf.', ', SVP, Volketswil.', ', GLP, Dübendorf.',
    ', SVP, Uetikon am See.']

records = []
notfound = []

# Loop file
for i, row in df.iterrows():

    # Create Name and load
    f = Path('../export/extracts/%s' % row['_filename']).with_suffix('.json')
    data = json.load(open(f))

    filename = Path(f).stem

    for msg in data:
        #print(msg)

        # Replace double space
        msg = msg.replace('  ', ' ')

        if msg.strip() == '':
            continue

        # Remove Strings
        for r in remove:
            msg = msg.replace(r, '')

        msg = msg.strip()

        # Extract Name
        r1 = re.findall(r"[^(:]*", msg)
        if len(r1) > 0:
            name = r1[0].strip()
            r = df_raete[df_raete['_name'] == name.lower()]
            # If name not found, add it to a list for manual check
            if len(r) == 0:
                notfound.append({
                    'name': name,
                    'f': filename,
                })
                
            else:
                # Name found!

                # Clean text
                text = msg
                text = text.replace('-\n', '')
                text = text.replace('\n', ' ')
                text = text.replace('  ', ' ')
                text = text.strip()

                text = re.findall(r".*:(.*)", text)
                if len(text) > 0:
                    text = text[0].strip()

                    records.append({
                        'name': r.iloc[0]['name'],
                        'vorname': r.iloc[0]['vorname'],
                        'text': text,
                        'f': filename
                    })
        else:
            # No Name found
            print('!! No Name found in:\n%s' % msg)

print("finito")

finito


## Join with dataframe and save

In [14]:
df_party = pd.read_csv(Path('../export/mitglieder.csv'))
df_party['_join'] = df_party.apply(lambda row: "%s_%s" % (row['name'], row['vorname']), axis=1)

df_votum = pd.DataFrame(records)
df_votum['_join'] = df_votum.apply(lambda row: "%s_%s" % (row['name'], row['vorname']), axis=1)

# No join, because could be multiple Partys per name (if they switched). Loop and find party

for i, row in df_votum.iterrows():
    xx

#df_votum = df_votum.merge(df_party[['_join', 'geschlecht', 'jahrgang', 'partei']], on = '_join')

df_votum.tail()

,name_x,vorname_x,text,f,_join,name_y,vorname_y,geschlecht,jahrgang,partei,start,end
32701,Kreuzer,Gregor,Die Ombudsstelle des Kantons Zürich ist eine w...,2021-09-20-609d7a42c5024983988d5cf4d2d31003-332,Kreuzer_Gregor,Kreuzer,Gregor,m,1980,GLP,2021-03-29T00:00:00.000,2021-03-22T00:00:00.000
32702,Kreuzer,Gregor,Viele Massnahmen der Pandemie bedeuteten die P...,2021-09-20-609d7a42c5024983988d5cf4d2d31003-332,Kreuzer_Gregor,Kreuzer,Gregor,m,1980,GLP,2021-03-29T00:00:00.000,2021-03-22T00:00:00.000
32703,Te,Yiea Wey,Ich gelobe es.,2021-05-03-0ba6beecddc74261b21bccc822a4b635-332,Te_Yiea Wey,Te,Yiea Wey,m,1983,FDP,2021-05-03T00:00:00.000,2021-04-26T00:00:00.000
32704,Wyss,Nicole,Ich gelobe es.,2021-09-06-f12519a320d741a29754b5a97e9afc33-332,Wyss_Nicole,Wyss,Nicole,w,1976,AL,2021-09-06T00:00:00.000,2019-05-05T00:00:00.000
32705,Wyss,Nicole,Man traut den Veranstaltenden nicht. Dies ist ...,2021-09-06-f12519a320d741a29754b5a97e9afc33-332,Wyss_Nicole,Wyss,Nicole,w,1976,AL,2021-09-06T00:00:00.000,2019-05-05T00:00:00.000


In [8]:
df_party.head()

,name,vorname,geschlecht,jahrgang,partei,start,end
0,Zaugg-Brüllmann,Marlies,w,1946,FDP,2005-11-07T00:00:00.000,2011-05-08T00:00:00.000
1,Stofer,Judith Anna,w,1959,AL,2011-05-09T00:00:00.000,2011-05-08T00:00:00.000
2,Walti,Rahel,w,1972,GLP,2008-02-25T00:00:00.000,2013-02-23T00:00:00.000
3,Schmid,Stefan,m,1976,SVP,2015-11-03T00:00:00.000,2013-02-23T00:00:00.000
4,Stampfli,Michael,m,1988,SP,2014-03-31T00:00:00.000,2015-05-17T00:00:00.000


In [5]:

df_votum.to_csv(Path('../export/df_votum.csv'), index=False)

,name,vorname,text,f
0,Amacker,Bruno,Ich gelobe es.,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332
1,Vogt,Hans-Ueli,Die Redaktionskommission hat am Ende des verga...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332
2,Häring,Hans Peter,Die EDU hat sich für die Variante «100 Franken...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332
3,Vogt,Hans-Ueli,Auch bei dieser Vorlage hat die Redaktionskomm...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332
4,Gutmann,Eva,Das vorliegende Postulat wurde vor fünf Jahren...,2013-01-14-d47a280e1e704445ba0db16a80b53ce6-332
